In [4]:
#最简单的协程程序
def simpleCoroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)
    
myCoro = simpleCoroutine()
myCoro.send(None)
myCoro.send(42)

-> coroutine started
-> coroutine received: 42


StopIteration: 

In [5]:
# 不可以在没有预激协程的时候调用参数不为None的.send(),为None时也可以预激协程

In [6]:
#可以通过from inspect import getgeneratorstate 查看协程状态

In [7]:
# 使用协程计算移动平均值
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [8]:
a = averager()
next(a)

In [10]:
for i in range(11):
    print(a.send(i))

4.090909090909091
3.8333333333333335
3.6923076923076925
3.642857142857143
3.6666666666666665
3.75
3.8823529411764706
4.055555555555555
4.2631578947368425
4.5
4.761904761904762


In [11]:
a.close()

In [13]:
# 不预激的协程没什么用，预激协程的装饰器
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [14]:
a = averager()
for i in range(11):
    print(a.send(i))

0.0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0


In [15]:
## 让协程返回值
# 不预激的协程没什么用，预激协程的装饰器
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return count, average

In [16]:
a = averager()
for i in range(11):
    print(a.send(i))
a.send(None)

0.0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0


StopIteration: (11, 5.0)

In [17]:
#捕获异常返回值
a = averager()
for i in range(11):
    print(a.send(i))
try:
    a.send(None)
except StopIteration as e:
    r = e.value
r

0.0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0


(11, 5.0)

In [38]:
# 使用yield from
from collections import namedtuple
from functools import wraps

Result = namedtuple('Result', ('count average'))

#子生成器
def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer


def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

#委托生成器
def grouper(results, key):
    while True:
        results[key] = yield from averager()
        
#客户端（调用者）
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        print('group', group)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
    report(results)
    
#输出报告
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(result.count, group, result.average, unit))
        
data = {
    'girls;kg': [40.9, 41, 42, 35],
    'girls;m': [140.9, 141, 142, 135],
    'boys;kg': [50.9, 51, 52, 45],
    'boys;m': [240.9, 241, 242, 235],
}

main(data)

group <generator object grouper at 0x7f8ad4a17ca8>
group <generator object grouper at 0x7f8ad4a17c50>
group <generator object grouper at 0x7f8ad4a17fc0>
group <generator object grouper at 0x7f8ad4a17ca8>
 4 boys  averaging 49.73kg
 4 boys  averaging 239.72m
 4 girls averaging 39.73kg
 4 girls averaging 139.72m


In [65]:
# 出租车运营仿真
from collections import namedtuple
from functools import wraps
import queue
from random import randint

Event = namedtuple('Event', ('time proc action'))

# taxiProcess 协程， 实现各辆出租车的活动
def taxiProcess(ident, trips, startTime = 0):
    """
    每次状态改变后忙吧控制权交给仿真器
    ident: 出租车编号
    trips: 行程数量
    startTime: 出发时间
    """
    time = yield Event(startTime, ident, '离开车库')
    for i in range(trips):
        time = yield Event(time, ident, '乘客上车')
        time = yield Event(time, ident, '乘客下车')
    yield Event(time, ident, '工作结束，回家啦')
    
DEPARTUE_INTERVAL = 5
numTaxis = 3
taxis = {i: taxiProcess(i, (i + 1) * 2, i * DEPARTUE_INTERVAL) for i in range(numTaxis)}

# 随机程序
def computeDuration(previousAction):
    return randint(1, 6)

#仿真控制台
class Simulator:
    
    def __init__(self, procsMap):
        self.events = queue.PriorityQueue()
        self.procs = dict(procsMap)
    
    def run(self, endTime):
        """排定并显示时间，直到时间结束"""
        #排定各辆出租车的第一个时间
        for _, proc in sorted(self.procs.items()):
            firstEvent = next(proc)
            self.events.put(firstEvent)
            
        #这个仿真系统的循环
        simTime = 0
        while simTime < endTime:
            if self.events.empty():
                print("*** end of events ***")
                break
            
            currentEvent = self.events.get()
            simTime, procId, previousAction = currentEvent
            print('  ' * procId + 'taxi:', procId, currentEvent)
            activeProc = self.procs[procId]
            nextTime = simTime + computeDuration(previousAction)
            try:
                nextEvent = activeProc.send(nextTime)
            except StopIteration as e:
                del self.procs[procId]
            else:
                self.events.put(nextEvent)
        else:
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.foramt(self.events.qsize()))

In [66]:
s = Simulator(taxis)

In [67]:
s.run(100)

taxi: 0 Event(time=0, proc=0, action='离开车库')
taxi: 0 Event(time=2, proc=0, action='乘客上车')
taxi: 0 Event(time=3, proc=0, action='乘客下车')
  taxi: 1 Event(time=5, proc=1, action='离开车库')
taxi: 0 Event(time=7, proc=0, action='乘客上车')
  taxi: 1 Event(time=10, proc=1, action='乘客上车')
    taxi: 2 Event(time=10, proc=2, action='离开车库')
taxi: 0 Event(time=11, proc=0, action='乘客下车')
  taxi: 1 Event(time=12, proc=1, action='乘客下车')
taxi: 0 Event(time=14, proc=0, action='工作结束，回家啦')
    taxi: 2 Event(time=15, proc=2, action='乘客上车')
  taxi: 1 Event(time=17, proc=1, action='乘客上车')
  taxi: 1 Event(time=18, proc=1, action='乘客下车')
    taxi: 2 Event(time=19, proc=2, action='乘客下车')
    taxi: 2 Event(time=21, proc=2, action='乘客上车')
  taxi: 1 Event(time=22, proc=1, action='乘客上车')
  taxi: 1 Event(time=25, proc=1, action='乘客下车')
    taxi: 2 Event(time=26, proc=2, action='乘客下车')
  taxi: 1 Event(time=27, proc=1, action='乘客上车')
    taxi: 2 Event(time=30, proc=2, action='乘客上车')
  taxi: 1 Event(time=33, proc=1, action='